In [42]:
# Cell 1: Import necessary libraries
import numpy as np
from scipy.ndimage import zoom


In [43]:
# Cell 2: Define the size of the LUT
LUT_SIZE = 256

# Define the size of the small and large LUTs
SMALL_LUT_SIZE = 128
LARGE_LUT_SIZE = 256

In [44]:
# Create an identity LUT
lut = np.zeros((LUT_SIZE, LUT_SIZE, LUT_SIZE, 3))

# Fill the LUT with the identity mapping
for r in range(LUT_SIZE):
    for g in range(LUT_SIZE):
        for b in range(LUT_SIZE):
            lut[r, g, b] = [r/LUT_SIZE, g/LUT_SIZE, b/LUT_SIZE]

In [45]:
def interpolate_lut_trilinear(color, lut, LUT_SIZE):
    
    # Get color components
    r = color[0] / (2**(8-LUT_SIZE))
    g = color[1] / (2**(8-LUT_SIZE))
    b = color[2] / (2**(8-LUT_SIZE))

    # Convert the inputs to integer indices
    r1 = np.floor(r).astype(int)
    r2 = min(r1 + 1, lut.shape[0]-1)
    g1 = np.floor(g).astype(int)
    g2 = min(g1 + 1, lut.shape[1]-1)
    b1 = np.floor(b).astype(int)
    b2 = min(b1 + 1, lut.shape[2]-1)

    # Get the 8 corner values from the LUT
    f1 = lut[r1, g1, b1]
    f2 = lut[r2, g1, b1]
    f3 = lut[r1, g2, b1]
    f4 = lut[r2, g2, b1]
    f5 = lut[r1, g1, b2]
    f6 = lut[r2, g1, b2]
    f7 = lut[r1, g2, b2]
    f8 = lut[r2, g2, b2]

    # Perform trilinear interpolation for each color component
    interpolated_color = np.empty(3)
    for i in range(3):
        interpolated_color[i] = f1[i] + \
                                (r - r1) * (f2[i] - f1[i]) + \
                                (g - g1) * (f3[i] - f1[i]) + \
                                (b - b1) * (f5[i] - f1[i]) + \
                                (r - r1) * (g - g1) * (f4[i] - f1[i]) + \
                                (r - r1) * (b - b1) * (f6[i] - f1[i]) + \
                                (g - g1) * (b - b1) * (f7[i] - f1[i]) + \
                                (r - r1) * (g - g1) * (b - b1) * (f8[i] - f1[i])
    
    return interpolated_color

In [46]:
# Cell 5: Test the function with different LUT resolutions
for ADDR_WIDTH in range(3, 8):
    # Reduce the LUT
    reduced_lut = lut[::2**ADDR_WIDTH, ::2**ADDR_WIDTH, ::2**ADDR_WIDTH, :]

    # Generate random points in the original LUT
    points = np.random.uniform(0, lut.shape[0]-1, size=(100, 3))

    # Calculate the interpolation error
    errors = []
    for point in points:
        x, y, z = np.floor(point).astype(int)
        # Scale down the coordinates for the reduced LUT
        x2, y2, z2 = (x // 2**ADDR_WIDTH, y // 2**ADDR_WIDTH, z // 2**ADDR_WIDTH)
        # Ensure the coordinates are within the bounds of the reduced_lut
        x2 = min(max(x2, 0), reduced_lut.shape[0]-1)
        y2 = min(max(y2, 0), reduced_lut.shape[1]-1)
        z2 = min(max(z2, 0), reduced_lut.shape[2]-1)
        interpolated_value = interpolate_lut(reduced_lut, x2, y2, z2)
        true_value = lut[x, y, z]
        error = np.abs(interpolated_value - true_value)
        errors.append(error)

    print(f"ADDR_WIDTH={ADDR_WIDTH}, Average error={np.mean(errors)}")

ADDR_WIDTH=3, Average error=0.012955729166666667
ADDR_WIDTH=4, Average error=0.028151041666666668
ADDR_WIDTH=5, Average error=0.0601171875
ADDR_WIDTH=6, Average error=0.11848958333333333
ADDR_WIDTH=7, Average error=0.24888020833333332


In [47]:
# Load the existing LUT
small_lut = np.loadtxt(f"reinhard_lut_int_{SMALL_LUT_SIZE}.txt", dtype=int)
small_lut = small_lut.reshape((SMALL_LUT_SIZE, SMALL_LUT_SIZE, SMALL_LUT_SIZE, 3)) / 255

# Test the function with different LUT resolutions
for ADDR_WIDTH in range(3, 8):
    # Reduce the LUT
    reduced_lut = small_lut[::2**ADDR_WIDTH, ::2**ADDR_WIDTH, ::2**ADDR_WIDTH, :]

    # Interpolate the reduced LUT to the original resolution
    interpolated_lut = np.zeros_like(small_lut)
    for x in range(small_lut.shape[0]):
        for y in range(small_lut.shape[1]):
            for z in range(small_lut.shape[2]):
                x2, y2, z2 = (x // 2**ADDR_WIDTH, y // 2**ADDR_WIDTH, z // 2**ADDR_WIDTH)
                x2 = min(max(x2, 0), reduced_lut.shape[0]-1)
                y2 = min(max(y2, 0), reduced_lut.shape[1]-1)
                z2 = min(max(z2, 0), reduced_lut.shape[2]-1)
                interpolated_lut[x, y, z] = interpolate_lut(reduced_lut, x2, y2, z2)

    # Calculate the relative error for each color component
    epsilon = 1e-7  # small constant to avoid division by zero
    error = np.abs((interpolated_lut - small_lut) / (small_lut + epsilon))

    # Calculate the average error for each color component
    avg_errors = np.mean(error, axis=(0, 1, 2))
    print(f"ADDR_WIDTH={ADDR_WIDTH}, Average error per component={avg_errors}")

ADDR_WIDTH=3, Average error per component=[0.12716631 0.12716631 0.12716631]
ADDR_WIDTH=4, Average error per component=[0.23155353 0.23155353 0.23155353]
ADDR_WIDTH=5, Average error per component=[0.39615052 0.39615052 0.39615052]
ADDR_WIDTH=6, Average error per component=[0.64365267 0.64365267 0.64365267]
ADDR_WIDTH=7, Average error per component=[0.99218696 0.99218696 0.99218696]


: 